# COVID-19 UK - Delta Variant Surge

![](./Bolton_Neighbours_Delta_Cases.jpg)
![](./Bolton_Neighbours_Delta_Deaths.jpg)

From the beginning of May 2021, in the UK there has been a growing number of cases of the 'Delta' B.1.617.2 variant of SARS-CoV-2, which was first identified in India.

The local boroughs that seem to have been initially most affected are Bolton (Greater Manchester) and Blackburn with Darwen (Lancashire). Therefore this is a short notebook tracking the cases in these boroughs and the adjoining boroughs to monitor the extent of the contagion. It will be interesting to see whether the efficacy of the vaccine and the success of the UK roll-out will be enough to contain this outbreak of a novel variant, and/or whether the case-fatality-ratio is significantly reduced.

The data is from the [UK COVID-19 Dataset](https://www.kaggle.com/vascodegama/uk-covid19-data) so unfortunately the cases represent ALL COVID-19 confirmed cases regardless of the variant. 

There is also some redundancy in the code (and it's a bit messy!) because I have forked this from my own extended work on COVID-19 in the UK, which can be found here: [UK COVID-19: Cases, Deaths, Vaccines, Regions](https://www.kaggle.com/vascodegama/uk-covid-19-cases-deaths-vaccines-regions)

Any questions/comments let me know!

# Imports
______________________________________________________________________________

In [ ]:
# General Imports
import numpy as np 
import pandas as pd      
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime
from datetime import date, timedelta
import scipy.stats

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load Data and Create DataFrames
______________________________________________________________________________

In [ ]:
# Load CSV data to pandas DataFrames

UK_TOTAL_DATA = pd.read_csv('../input/uk-covid19-data/UK_National_Total_COVID_Dataset.csv', index_col='date',parse_dates=True) 
DEVOLVED_NATION_DATA = pd.read_csv('../input/uk-covid19-data/UK_Devolved_Nations_COVID_Dataset.csv', index_col='date',parse_dates=True)
ENGLAND_REGIONS_DATA = pd.read_csv('../input/uk-covid19-data/England_Regions_COVID_Dataset.csv', index_col='date',parse_dates=True)
UK_LOCAL_AUTHORITY_DATA = pd.read_csv('../input/uk-covid19-data/UK_Local_Authority_UTLA_COVID_Dataset.csv', index_col='date',parse_dates=True)
POPULATION_DATA = pd.read_csv('../input/uk-covid19-data/NEW_Official_Population_Data_ONS_mid-2019.csv',index_col='Unnamed: 0')

UK_AND_NATIONS = pd.concat([UK_TOTAL_DATA,DEVOLVED_NATION_DATA])
ENGLAND_AND_REGIONS = pd.concat([DEVOLVED_NATION_DATA[DEVOLVED_NATION_DATA['areaName'] == 'England'],ENGLAND_REGIONS_DATA])
UK_AND_UTLAS = pd.concat([UK_TOTAL_DATA,UK_LOCAL_AUTHORITY_DATA])

In [ ]:
# Add population density data to UK_LOCAL_AUTHORITY_DATA dataframe
mapping = dict(POPULATION_DATA[['areaCode', 'Population Density (per sq. km)']].values)
UK_LOCAL_AUTHORITY_DATA['Population Density (per sq. km)'] = UK_LOCAL_AUTHORITY_DATA['areaCode'].map(mapping)

# Add population data to UK_LOCAL_AUTHORITY_DATA dataframe
MAPPING_POP = dict(POPULATION_DATA[['areaCode', 'Population']].values)
UK_LOCAL_AUTHORITY_DATA['Population'] = UK_LOCAL_AUTHORITY_DATA['areaCode'].map(MAPPING_POP)

# Add population data to UK_AND_NATIONS dataframe - '.title()' method used to convert country names from all caps so they match with COVID-19 DataFrame
NATION_MAPPING_POP = dict(POPULATION_DATA[['areaName', 'Population']].values)
NATION_MAPPING_POP_fix = dict(zip([i.title() for i in NATION_MAPPING_POP.keys()], [i for i in NATION_MAPPING_POP.values()]))

UK_AND_NATIONS['Population'] = UK_AND_NATIONS['areaName'].map(NATION_MAPPING_POP_fix)

In [ ]:
# Calculate 7-day averages for new deaths and new deaths per 100,000 people

SEVEN_DAY_AVERAGES = pd.Series(dtype='float64')

for i in UK_AND_NATIONS['areaName'].unique():
    SEVEN_DAY_AVERAGES = SEVEN_DAY_AVERAGES.append(UK_AND_NATIONS[UK_AND_NATIONS["areaName"] == i]['newDeaths28DaysByDeathDate'].rolling(window=7).mean())
    
UK_AND_NATIONS['New_Deaths_7_day_average'] = SEVEN_DAY_AVERAGES

# Create new 7 day rolling average deaths per 100,000 people

UK_AND_NATIONS['New_Deaths_7_day_average_per_100,000'] = (UK_AND_NATIONS['New_Deaths_7_day_average']*100000/
                                                                   UK_AND_NATIONS['Population'])

# Calculate 7-day averages for new cases 

AVERAGES_CASES = pd.Series(dtype='float64')

for i in UK_AND_NATIONS['areaName'].unique():
    AVERAGES_CASES = AVERAGES_CASES.append(UK_AND_NATIONS[UK_AND_NATIONS["areaName"] == i]['newCasesByPublishDate'].rolling(window=7).mean())
    
UK_AND_NATIONS['New_Cases_7_day_average'] = AVERAGES_CASES


# Create %population first-dose vaccinated

UK_AND_NATIONS['People_Vaccinated_First_Dose_Percent'] = UK_AND_NATIONS['cumPeopleVaccinatedFirstDoseByPublishDate']*100/UK_AND_NATIONS['Population'].fillna(0)

In [ ]:
SEVEN_DAY_AVERAGES = pd.Series(dtype='float64')

for i in UK_LOCAL_AUTHORITY_DATA['areaName'].unique():
    SEVEN_DAY_AVERAGES = SEVEN_DAY_AVERAGES.append(UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"] == i]['newDeaths28DaysByDeathDate'].rolling(window=7).mean())
    
UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average'] = SEVEN_DAY_AVERAGES

In [ ]:
# Create new 7 day rolling average deaths per 100,000 people

UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average_per_100,000'] = (UK_LOCAL_AUTHORITY_DATA['New_Deaths_7_day_average']*100000/
                                                                   UK_LOCAL_AUTHORITY_DATA['Population'])

# Daily New Cases in Bolton, Blackburn and Adjoining Boroughs

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTXlAA8pNyaLJ_kD47GyhYdTrWK8QUGMR11qA&usqp=CAU)

(Colours in image are not data driven)

In [ ]:
# Set-up DF for boroughs bordering Bolton since 1st May
UK_LOCAL_AUTHORITY_DATA['New_Cases_per_100k'] = UK_LOCAL_AUTHORITY_DATA['newCasesByPublishDate']*100000/UK_LOCAL_AUTHORITY_DATA['Population']

bolton_neighbours = ['Bolton', 'Blackburn with Darwen', 'Wigan', 'Salford', 'Bury','Lancashire']

BOLTON_NEIGHBOURS_SINCE_MAY = UK_LOCAL_AUTHORITY_DATA[UK_LOCAL_AUTHORITY_DATA["areaName"].isin(bolton_neighbours)].loc['2021-05-01':UK_AND_NATIONS.index[-4].strftime("%Y-%m-%d")]

In [ ]:
f, axs = plt.subplots(figsize=(15,6))



sns.lineplot(data=BOLTON_NEIGHBOURS_SINCE_MAY,
             x=BOLTON_NEIGHBOURS_SINCE_MAY.index,
             y='New_Cases_per_100k',
            hue="areaName",ax=axs)
axs.grid(linestyle=':', linewidth='0.5')
axs.set_title('Confirmed New Cases of COVID-19 (All Variants) per 100,000 Residents')
axs.set_xlim(date.fromisoformat('2021-05-01'),UK_AND_NATIONS.index[-4])
axs.set_ylim(0,BOLTON_NEIGHBOURS_SINCE_MAY['New_Cases_per_100k'].max()*1.05)
axs.grid(linestyle=':', linewidth='0.5')


plt.savefig('Bolton_Neighbours_Delta_Cases.jpg', dpi=300)
plt.show

In [ ]:
BOLTON_NEIGHBOURS_SINCE_MAY.columns

In [ ]:
f, axs = plt.subplots(figsize=(15,6))



sns.lineplot(data=BOLTON_NEIGHBOURS_SINCE_MAY,
             x=BOLTON_NEIGHBOURS_SINCE_MAY.index,
             y='New_Deaths_7_day_average_per_100,000',
            hue="areaName",ax=axs)
axs.grid(linestyle=':', linewidth='0.5')
axs.set_title('New Daily Deaths per 100,000 People (7-day average)')
axs.set_xlim(date.fromisoformat('2021-05-01'),UK_AND_NATIONS.index[-4])
axs.set_ylim(0,BOLTON_NEIGHBOURS_SINCE_MAY['New_Deaths_7_day_average_per_100,000'].max()*1.05)
axs.grid(linestyle=':', linewidth='0.5')


plt.savefig('Bolton_Neighbours_Delta_Deaths.jpg', dpi=300)
plt.show

# UK Total New Confirmed Cases and Rate of Change (for context)
______________________________________________________________________________

In [ ]:
# Plot Daily New Deaths vs Percent of Population Vaccinated (First-Dose)

NEW_DEATHS_GRAD = np.gradient(UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom']["New_Cases_7_day_average"].rolling(7).mean())

f, ax1 = plt.subplots(figsize=(15,6))
sns.lineplot(data=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'],
             x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y="New_Cases_7_day_average")
plt.title('New Daily Confirmed Cases (7-day Rolling Average) and Rate-of-Change for United Kingdom')
ax1.grid(linestyle=':', linewidth='0.5')
ax1.set_xlim(date.fromisoformat('2020-08-01'),UK_AND_NATIONS.index[-2])
ax1.set_ylim(0)
#ax.legend(ncol=2,title=None,loc=(0.45,0.03))


# Full UK Lockdown 1 Text
TEXT = [('2020-03-22','First UK National Lockdown'),
        ('2020-05-25','NHS Test & Trace'),
        ('2020-06-24','First Local Lockdown'),
        ('2020-10-10','3-Tier System'),
        ('2020-10-19','Wales Lockdown'),
        ('2020-11-01','England Lockdown'),
        ('2020-11-28','Pfizer Vaccine Approval'),
        ('2020-12-26','Oxf/AstZen Vaccine Approval')
       ]

for i,j in TEXT[3:]: 
    ax1.text(date.fromisoformat(i), UK_AND_NATIONS['New_Cases_7_day_average'].max()*1.04, j, rotation=90,va='top')
# for i,j in TEXT[6:]: 
#     ax.text(date.fromisoformat(i), 3, j, rotation=90)

# Vertical Lines

# ax1.axvline(date.fromisoformat('2020-03-23'), ls='--', color='black')
# ax1.axvline(date.fromisoformat('2020-05-28'), ls='--', color='black')
# ax1.axvline(date.fromisoformat('2020-06-29'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-14'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-10-23'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-11-05'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-02'), ls='--', color='black')
ax1.axvline(date.fromisoformat('2020-12-30'), ls='--', color='black')

# Second axis (Vaccine introduction)

ax2 = ax1.twinx()
color = 'magenta'
ax2.set_ylabel('Rate of Change of New Daily Cases', color=color)  # we already handled the x-label with ax1
sns.lineplot(x=UK_AND_NATIONS[UK_AND_NATIONS['areaName']=='United Kingdom'].index,
             y=NEW_DEATHS_GRAD,color=color,linestyle='--',ax=ax2)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(-3500,3500)
plt.savefig('UK_Cases_and_Rate_of_Change.jpg',dpi=300)
plt.show
